In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

In [2]:
pip install pandas matplotlib bokeh

Note: you may need to restart the kernel to use updated packages.


In [3]:
from __future__ import print_function
import pandas as pd
from datetime import datetime
import csv
from bokeh.models import ColumnDataSource, HoverTool, Legend
from bokeh.plotting import figure, show
from bokeh.palettes import Category10
import itertools
import random 
import string

import matplotlib.dates as mdates
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display
from bokeh.io import output_notebook

def color_gen():
    yield from itertools.cycle(Category10[10])
color = color_gen()

def plot_lines_multi(df,lw=2,pw=1200,ph=400,t_str="save,pan,box_zoom,reset,wheel_zoom",t_loc='above'):
    '''...
    '''
    source = ColumnDataSource(df)
    col_names = source.column_names
    p = figure(x_axis_type="linear",plot_width=pw, plot_height=ph,toolbar_location=t_loc, tools=t_str, output_backend="webgl")
    p_dict = dict()
    lastRtdName = ""
    for col, c, col_name in zip(df.columns,color,col_names):
        chunk = col_name.split('_')
        rtd = chunk[len(chunk)-1]
        if (col_name == 'index' or col_name == 'timeRef'): #rtd == "total" or rtd == "mean" or
            lastRTD = rtd
            continue
    
        if (rtd != lastRtdName and  len(p_dict.items()) > 1):
            legend = Legend(items=[(x, [p_dict[x]]) for x in p_dict])
            p.add_layout(legend)#, 'right')#,'right')
            p.legend.click_policy="hide"
            show(p)

            p = figure(x_axis_type="linear",plot_width=pw, plot_height=ph,toolbar_location=t_loc, tools=t_str, output_backend="webgl")
            p_dict = dict()
           
        p_dict[col_name] = p.line('timeRef', col_name, source=source, color=c,line_width=lw)
        p.add_tools(HoverTool(
            renderers=[p_dict[col_name]],
            tooltips=[('timeRef','@{timeRef}ms'),(col_name, f'@{col}')],
        ))            
        lastRtdName = rtd
    legend =Legend(items=[(x, [p_dict[x]]) for x in p_dict])
    p.add_layout(legend)#,'right')
    p.legend.click_policy="hide"
    show(p)
    

def f(offsetFFT2,offsetSPW_Acquisition2,samplingRate):
    df = pd.read_csv("Use_case_DoubleCamera_Safety_Priority.ATIPPIC0.executable"+str(offsetFFT2)+".0_"+str(offsetSPW_Acquisition2)+".0/Use_case_DoubleCamera_Safety.ATIPPIC.executable30.0_30.0.csv",sep=',',
                    skiprows=lambda i: i % (100/samplingRate) != 0)#, index_col=False)
    df = df.drop(['HP2_latence_total', 'HP23_latence_total', 'CortexToDDR_latence_total', 'HP2_latence_mean', 'HP23_latence_mean', 'CortexToDDR_latence_mean'], axis=1)
    print('please wait while loading the 3 graphs...')
    output_notebook()
    df['timeRef'] = df['timeRef'].apply(lambda x: x*0.003125)
    df.set_index(df['timeRef'])
    plot_lines_multi(df)
    return
style = {'description_width': 'initial'}
graph_throughputs = interactive(f, 
                                offsetFFT2=widgets.IntSlider(min=100.0,max=300.0,step=10.0,value=200,continuous_update=False, style=style),
                                offsetSPW_Acquisition2=widgets.IntSlider(min=0.0,max=100.0,step=10.0,value=0,continuous_update=False, style=style),
                                samplingRate=widgets.Dropdown(options=[100, 50, 10],value=10,description='sampling rate:',disabled=False, style=style)

);
display(graph_throughputs)


interactive(children=(IntSlider(value=200, continuous_update=False, description='offsetFFT2', max=300, min=100…